In [2]:
import seqio

/fsx/home-mistobaan/mambaforge/envs/flan/lib/python3.8/site-packages/flax/core/frozen_dict.py:169: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(
2023-03-27 17:44:25.660801: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from flan.v2 import mixtures

2023-03-27 17:44:42.540339: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [4]:
seqio.MixtureRegistry.add(
    't0_submix',
    tasks=[
        ('t0_zsopt', 1),      # mixing weight = 25%
        # ('t0_fsopt', 1),      # mixing weight = 25%
        # ('t0_zsnoopt', 1),    # mixing weight = 25%
        # ('t0_fsnoopt', 1),    # mixing weight = 25%
    ])

In [5]:
# !wget https://curl.se/ca/cacert-2023-01-10.pem -O ~/cacert.pem

In [6]:
# https://curl.se/docs/caextract.html
%env CURL_CA_BUNDLE /fsx/home-mistobaan/cacert.pem

env: CURL_CA_BUNDLE=/fsx/home-mistobaan/cacert.pem


In [7]:
import tensorflow_datasets as tdfs

selected_mixture = seqio.get_mixture_or_task('t0_submix')

dataset = selected_mixture.get_dataset(
    sequence_length={"inputs": 4096, "targets": 4096},
    split=tdfs.Split.TRAIN,
    num_epochs=1,
    shuffle=False,
    copy_pretokenized=True,
    passthrough_features=["_template_idx", "_task_source", "_task_name", "_template", "_template_type"]    
)

2023-03-27 17:44:44.524953: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Instructions for updating:
Use `tf.data.Dataset.sample_from_datasets(...)`.


Instructions for updating:
Use `tf.data.Dataset.sample_from_datasets(...)`.


In [17]:
it = iter(dataset)

In [9]:
item = _

In [16]:
(
    item['inputs_pretokenized'].numpy().decode(),
    item['targets_pretokenized'].numpy().decode(),
    item['_template_idx'].numpy(),
    item['_template_type'].numpy().decode(),
    item['_task_name'].numpy().decode(),
    item['_task_source'].numpy().decode()
)

('Q: Where are most of the organs contained in insects?   Read this paragraph and choose the correct option from the provided answers:  Like other arthropods, insects have three body segments and many jointed appendages. The abdomen contains most of the internal organs. Six legs are attached to the thorax. There are several appendages on the insect’s head:.   Choices:  - head  - appendages  - thorax  - the abdomen   A:\n',
 'the abdomen',
 0,
 'zs_opt',
 'sciq_Multiple_Choice_Question_First',
 'P3')

In [ ]:
save_data = []
source_counter = defaultdict(lambda: 0)
NUM_SAMPLES = 100
# If you would like to take min(1 epoch, NUM_SAMPLES) then use dataset.take(NUM_SAMPLES)
# Or if you would like to gather a full epoch, simply `enumerate(dataset)` until completion.
for i, ex in enumerate(dataset.take(NUM_SAMPLES)):
    source_counter[ex["_task_source"].numpy()] += 1
    save_data.append((ex["inputs_pretokenized"].numpy().decode(),
                      ex["targets_pretokenized"].numpy().decode()))

print(f"Data Submixture Counts: {source_counter}")